In [ ]:
from enum import Enum

class QueryType(str, Enum):
    LAW = "LAW"                # Case A
    DRAWING = "DRAWING"        # Case B
    COMPOSITE = "COMPOSITE"    # Case C
    UNSUPPORTED = "UNSUPPORTED"  # Case D

In [ ]:
LAW_KEYWORDS = [
    "법", "법령", "조례", "시행령", "시행규칙",
    "건축법", "주택법", "국토계획법", "면적기준"
]

DRAWING_KEYWORDS = [
    "도면", "평면도", "입면도", "단면도",
    "bay", "베이", "판상형", "타워형",
    "거실", "침실", "욕실", "수납"
]

In [ ]:
def keyword_filter(query: str) -> QueryType | None:
    q = query.lower()

    law_hit = any(k in q for k in LAW_KEYWORDS)
    drawing_hit = any(k in q for k in DRAWING_KEYWORDS)

    if law_hit and drawing_hit:
        return QueryType.COMPOSITE
    if law_hit:
        return QueryType.LAW
    if drawing_hit:
        return QueryType.DRAWING

    return None  # 애매한 경우 → LLM으로

In [ ]:
INTENT_CLASSIFIER_PROMPT = """
너는 사용자 질문을 분류하는 분류기다.
아래 네 가지 중 반드시 하나만 선택하라.

[분류 기준]
LAW:
- 법령, 조례, 규정, 기준, 허용 여부를 묻는 질문

DRAWING:
- 도면, 평면 구성, 공간 배치, 주거 타입, 설계 특성 질문

COMPOSITE:
- 법령과 도면 해석이 동시에 필요한 질문

UNSUPPORTED:
- 개인 의견, 철학, 잡담, 정보 부족, 답변 불가 질문

[출력 형식]
LAW | DRAWING | COMPOSITE | UNSUPPORTED

[질문]
{query}
"""

In [ ]:
from openai import OpenAI

client = OpenAI()

def classify_with_llm(query: str) -> QueryType:
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": "너는 질문 분류기다."},
            {
                "role": "user",
                "content": INTENT_CLASSIFIER_PROMPT.format(query=query)
            }
        ],
        temperature=0,
    )

    result = response.choices[0].message.content.strip()
    return QueryType(result)

In [ ]:
def classify_query(query: str) -> QueryType:
    # 1️. 키워드 가드
    keyword_result = keyword_filter(query)
    if keyword_result:
        return keyword_result

    # 2️. LLM 분류
    return classify_with_llm(query)

In [ ]:
def route_query(query: str):
    query_type = classify_query(query)

    if query_type == QueryType.LAW:
        return law_search(query)

    if query_type == QueryType.DRAWING:
        return drawing_search(query)

    if query_type == QueryType.COMPOSITE:
        return {
            "error": "복합 검색은 추후 지원 예정입니다."
        }

    return {
        "error": "해당 질문은 답변할 수 없습니다."
    }